# Notebook 12: Working with Physical Units

Underworld3 has built-in support for physical units throughout the modeling workflow. This makes your models easier to understand and helps catch dimensional errors early.

In this notebook you'll learn:
- Creating physical quantities (temperatures, velocities, viscosities)
- Converting between units
- Working with unit-aware arrays and coordinates
- Automatic unit tracking through derivatives

In [1]:
import nest_asyncio
nest_asyncio.apply()

import underworld3 as uw
import numpy as np
import sympy

## Creating Physical Quantities

Create quantities using `uw.units`, which provides access to the Pint units library. You can write units explicitly or use strings:

In [2]:
# Explicit units
plate_velocity = 5 * uw.units.cm / uw.units.year
mantle_temperature = 1500 * uw.units.K

# String notation (Pint can parse these)
mantle_depth = 2900 * uw.units("km")
mantle_viscosity = 1e21 * uw.units("Pa*s")

# Display (implicit - last line shows result)
mantle_temperature

<Quantity(1500, 'kelvin')>

In [3]:
# View all the quantities
plate_velocity, mantle_depth, mantle_viscosity

(<Quantity(5.0, 'centimeter / year')>,
 <Quantity(2900, 'kilometer')>,
 <Quantity(1e+21, 'pascal * second')>)

## Unit Conversions

Converting between compatible units is straightforward using the `.to()` method:

In [4]:
# Convert velocity to different units
plate_velocity.to("mm/year")

<Quantity(50.0, 'millimeter / year')>

In [5]:
# Convert to SI units
plate_velocity.to("m/s")

<Quantity(1.58440439e-09, 'meter / second')>

In [6]:
# Convert depth
mantle_depth.to("m")

<Quantity(2900000.0, 'meter')>

## Working with Mesh Coordinates

Create a simple mesh and variables to explore unit-aware operations:

In [ ]:
# Create a mesh
mesh = uw.meshing.UnstructuredSimplexBox(
    minCoords=(0.0, 0.0),
    maxCoords=(1000.0, 500.0),
    cellSize=50.0,
    qdegree=2,
)

mesh.dm.getCoordinateDM().getNumFields()

## Optional: Setting Up Reference Quantities

**Note**: You can use units in Underworld3 without setting reference quantities. However, setting them is **strongly recommended** for better numerical conditioning in solvers.

Reference quantities define the characteristic scales for your problem (temperature range, domain size, velocity scale). When set, they ensure your variables have proper scaling coefficients (e.g., `scaling_coefficient=1/1500` for a 1500 K temperature reference scale). This improves solver performance and conditioning.

**Without reference quantities**: Variables will use `scaling_coefficient=1.0`, which can lead to poorly conditioned systems if your physical values span many orders of magnitude.

**With reference quantities**: Variables automatically get appropriate scaling factors that normalize values to order-1 numbers, improving numerical stability.

In [ ]:
# Set up reference quantities for improved numerical scaling
# This is optional but recommended for better conditioning in solvers
model = uw.get_default_model()
model.set_reference_quantities(
    temperature_diff=uw.quantity(1500, "K"),  # Reference temperature range
    length=uw.quantity(1000, "m"),            # Reference length scale
    velocity=uw.quantity(0.01, "m/s")         # Typical velocity scale
)

In [8]:
# Create mesh variables with units
temperature = uw.discretisation.MeshVariable("T", mesh, 1, degree=2, units="K")
velocity = uw.discretisation.MeshVariable("u", mesh, 2, degree=2, units="m/s")

/Users/lmoresi/+Underworld/underworld-pixi-2/.pixi/envs/default/lib/python3.12/site-packages/underworld3/discretisation/persistence.py:111: UserWarning: 
Variable 'T' has units 'K' but no reference quantities are set.
Call model.set_reference_quantities() before creating variables with units.
Variable will use scaling_coefficient=1.0, which may lead to poor numerical conditioning.

  self._base_var = _BaseMeshVariable(
/Users/lmoresi/+Underworld/underworld-pixi-2/.pixi/envs/default/lib/python3.12/site-packages/underworld3/discretisation/persistence.py:111: UserWarning: 
Variable 'u' has units 'm/s' but no reference quantities are set.
Call model.set_reference_quantities() before creating variables with units.
Variable will use scaling_coefficient=1.0, which may lead to poor numerical conditioning.

  self._base_var = _BaseMeshVariable(


### Initializing Fields

Set up a simple temperature field:

In [9]:
# Get coordinate symbols
x, y = mesh.X

# Initialize temperature: T = 300 + 2.6*y (K)
temperature.array[...] = uw.function.evaluate(
    300.0 + 2.6 * y, temperature.coords
)

velocity.array[...] = uw.function.evaluate(
    sympy.Matrix([5.0, 0.0]), velocity.coords
).reshape(velocity.array.shape)

temperature.min(), temperature.max()

(300.0, 1600.0000000000005)

## Unit-Aware Operations

Variables with units support mathematical operations that preserve dimensional consistency:

In [10]:
# Get units from expressions
uw.get_units(temperature)

<Unit('kelvin')>

In [11]:
# Check dimensionality
uw.get_dimensionality(temperature)

<UnitsContainer({'[temperature]': 1})>

In [12]:
# Units work naturally with arithmetic
uw.get_units(temperature / velocity[0])

<Unit('second * kelvin / meter')>

### Automatic Unit Tracking Through Derivatives

When you take derivatives, units are tracked automatically:

In [13]:
# Derivative automatically has correct units
dTdy = temperature.diff(y)[0]

# The derivative has units!
uw.get_units(dTdy)

<Unit('kelvin')>

The derivative has units that make physical sense - temperature units divided by coordinate units.

## Computing Gradients with Projection

To get numerical gradient values at mesh nodes, use the Projection system:

In [14]:
# Create a variable to hold the gradient
gradT = uw.discretisation.MeshVariable(
    "gradT",
    mesh,
    1,
    degree=1,
    units="K/m",  # Specify units directly
)

# Project the derivative onto the mesh
proj = uw.systems.Projection(mesh, gradT)
proj.uw_function = temperature.diff(y)[0]
proj.solve()

gradT.min(), gradT.max()

/Users/lmoresi/+Underworld/underworld-pixi-2/.pixi/envs/default/lib/python3.12/site-packages/underworld3/discretisation/persistence.py:111: UserWarning: 
Variable 'gradT' has units 'K/m' but no reference quantities are set.
Call model.set_reference_quantities() before creating variables with units.
Variable will use scaling_coefficient=1.0, which may lead to poor numerical conditioning.

  self._base_var = _BaseMeshVariable(


(2.599999999999978, 2.6000000000000174)

In [15]:
# Gradient array is unit-aware
gradT.array[100, ...]

UnitAwareArray([[2.6]]), callbacks=0, units='kelvin / meter')

### Converting Gradient Units

In [16]:
# Get the gradient array (unit-aware)
gradT_array = gradT.array

# Convert to different units
gradT_km = gradT_array.to("K/km")

gradT_array[0, 0, 0], gradT_km[0, 0, 0]

AttributeError: 'SimpleMeshArrayView' object has no attribute 'to'

## Dimensional Analysis

Units help catch errors. For example, trying to add quantities with incompatible dimensions will fail:

In [ ]:
# This works - same dimensions
total_temperature = mantle_temperature + 100 * uw.units.K
total_temperature

In [ ]:
# This fails - incompatible dimensions
try:
    wrong = mantle_temperature + plate_velocity
except Exception as e:
    type(e).__name__, str(e)[:80]

## Summary

The units system in Underworld3:

- **Creation**: Use `uw.units` to create physical quantities
- **Conversion**: Use `.to(target_units)` to convert between compatible units
- **Derivatives**: Automatically get correct units (e.g., `temperature.diff(y)` has units K/m)
- **Type checking**: `uw.get_units()` and `uw.get_dimensionality()` let you inspect units
- **Error prevention**: Incompatible unit operations raise errors

Units make your code clearer and help catch errors early!

### What's Next?

**Notebook 13** introduces **non-dimensional scaling** and shows how to:
- Set up reference quantities for automatic scaling
- Solve problems in non-dimensional form for better numerical conditioning
- Convert easily between dimensional and non-dimensional representations

This makes working with multi-scale physical problems much more robust.

## Try It Yourself

Exercises to explore:

```python
# 1. Create different quantities
density = 3300 * uw.units("kg/m^3")
gravity = 9.81 * uw.units("m/s^2")
stress = density * gravity * mantle_depth

# 2. Check the units
uw.get_units(stress)

# 3. Convert to different units
stress.to("GPa")

# 4. Create a vector derivative
div_velocity = velocity[0].diff(x) + velocity[1].diff(y)
uw.get_units(div_velocity)
```